In [2]:
!pip install langchain
!pip install openai
!pip install pyPDF2
!pip install faiss-cpu
!pip install tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.5 MB/s eta 0:00:00


In [3]:
## importing all the libraries
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
import os


In [4]:
os.environ["OPENAI_API_KEY"] = ''


In [6]:
## Extracting text from PDF
reader = PdfReader("/content/wb-axp1.pdf")

In [7]:
raw_text = ""
for i, page in enumerate(reader.pages):
  text = page.extract_text()
  if text:
    raw_text += text
raw_text

'1964: AMERICAN EXPRESS\u2003/three.taboldstyle/five.taboldstyle\nFigure 4.1.was committing fraud. Allied went bankrupt, and the warehouse, when collecting the collateral, realized that the tanks that it had thought were /f_illed with valuable salad oil were instead /f_illed with seawater. With an estimated liability of up to /dollar.oldstyle/one.oldstyle/five.oldstyle/zero.oldstyle million, Allied and the American Express subsidiary /f_iled for bankruptcy protection. Whether American Express, the parent company, was liable was uncertain. Nevertheless, fearing damage to American Express’s reputation, CEO and President Howard Clark issued a statement that American Express felt morally bound to see that such liabili-ties were satis/f_ied./one.oldstyle/T_he word on the street was that American Express could face insol-vency. Its stock, which had been trading at /dollar.oldstyle/six.oldstyle/zero.oldstyle per share before the news, dropped to /dollar.oldstyle/three.oldstyle/five.oldstyle p

In [8]:
##Split the text into smaller number of chunks
textsplitter = CharacterTextSplitter(
    separator='\n',
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

In [9]:
texts = textsplitter.split_text(raw_text)
len(texts)

2

In [12]:
#User embeddings from OPENAI
embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_texts(texts, embeddings)

In [13]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
chain = load_qa_chain(OpenAI(), chain_type='stuff')

In [14]:
query="What was the scandal of American Express in 1964?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

" The scandal of American Express in 1964 was related to fraudulent activity in its subsidiary Allied Crude Vegetable Oil. Allied had taken out a loan with American Express and had used tanks of salad oil as collateral; however, the warehouse collecting the collateral found that the tanks were actually filled with seawater. This led to Allied and the American Express subsidiary filing for bankruptcy protection and fear of damage to American Express's reputation."

In [15]:
query="What was the main business of American Express that was important and profitable at the time?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The main business of American Express that was important and profitable at the time was travelers cheques.'

In [16]:
query="Write a summary of this chapter in 100 words."
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

" This chapter is about American Express' financial situation in 1964. It discusses the company's ten operating segments, the largest of which was the travelers cheques business. This business allowed customers to purchase paper checks at numerous locations before departure and have them accepted at banks and venues abroad. American Express' financials were very strong in the decade before 1964 and the company had delivered net earnings of $1.12 million on revenues of $1 billion that year. The company had an operating margin of 6% and a net profit margin of 1%, both suggesting good profitability. The chapter also explores the advantages of the travelers cheques business over the letter of credit and suggests that the company was well-run and that reputational damage from the fraud scandal was not likely to be permanent."